In [1]:
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing.image import image
import numpy as np
from keras.applications import VGG16
import cv2
import os

Using TensorFlow backend.


In [2]:
model=load_model("model1.h5")

In [14]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [15]:
def detect(gray,frame):
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    crop_img = frame
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        crop_img = frame[y: y + h, x: x + w]
        
    return frame,crop_img

In [20]:
def load_image(img):
    img = img.reshape(48,48,3)
    
    img_tensor = image.img_to_array(img) 
    img_tensor /= 255.  
    #print(img_tensor.shape)
    
    return img_tensor

In [21]:
def prediction(img_tensor): 
    #prediction = model.predict_classes(img_tensor.reshape(1,48,48,3))
    
    prediction = model.predict(img_tensor.reshape(1,48,48,3)) 
    prediction_classes = prediction.argmax(axis=-1)
    classes = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad","Surprise"]
    return (classes[prediction_classes[0]])

In [22]:
video_capture = cv2.VideoCapture('myvdo.mov')
fn=0
while True:
    _ , frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #cv2.imshow('Video',gray)
    canvas,crop_img = detect(gray,frame)
    no = str(fn)
    fn+=1
    #cv2.imwrite('my'+no+'.jpg',crop_img)
    crop_img=cv2.resize(crop_img,(48,48))
    name = prediction(load_image(crop_img))
    cv2.putText(frame, name, (0,225), cv2.FONT_HERSHEY_TRIPLEX, 1.0, (0,255,0), 2)
    cv2.imshow('Video',frame)
    #if(fn==3):
        #break;
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()